In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time 
import pymongo



In [2]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:

# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)


In [4]:
#dic to be return to app.py
mars_info = {}

# NASA Mars News

In [5]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [6]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [7]:
#prettify to analysze the html 
#print(soup.prettify())

In [8]:
#get news title

news_title = soup.find('div', class_='content_title').text
news_title

#second_news_title = soup.find_all('div', class_='content_title')[1].text

"NASA's InSight Places First Instrument on Mars"

In [9]:
#get paragraph text
news_p = soup.find('div', class_="rollover_description_inner").text
news_p

'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.'

In [10]:
mars_info['News_title'] = news_title
mars_info['paragraph'] = news_p

# JPL Mars Space Images - Featured Image

In [11]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called 

jpl_featured_space_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_featured_space_image_url)
time.sleep(2)

In [12]:
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)

In [13]:
browser.click_link_by_partial_text('more info')

In [14]:
html = browser.html
soup = bs(html, "html.parser")

In [15]:
featured_image_url = soup.find('img', class_="main_image")

In [16]:
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image_url['src']
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13911_hires.jpg


In [17]:
mars_info['mars_img'] = featured_image_url

# Mars Weather

In [18]:
mars_weather_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_twitter_url)

In [19]:
html = browser.html #doesn't work if i remove it, find out why?
mars_twitter = bs(html, 'html.parser')
#mars_weather = mars_twitter.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
mars_weather = mars_twitter.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text.strip()
mars_weather

'Sol 2283 (2019-01-07), high -9C/15F, low -71C/-95F, pressure at 8.25 hPa, daylight 06:44-18:55'

In [20]:
mars_info['mars_weather'] = mars_weather

# Mars Facts

In [21]:
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)

In [22]:
html = browser.html
mars_facts = bs(html, 'html.parser')

In [23]:
mars_facts_full_df = pd.read_html(mars_facts_url)
mars_facts_full_df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [24]:
mars_facts_df = pd.DataFrame(mars_facts_full_df[0])
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [25]:
mars_facts_df.columns = ['Characteristic','Data']
mars_facts_df = mars_facts_df.set_index("Characteristic")
mars_facts_df


,Data
Characteristic,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [26]:
mars_facts_html = mars_facts_df.to_html()
mars_facts_html = mars_facts_html.replace('\n', '')
mars_facts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Data</th>    </tr>    <tr>      <th>Characteristic</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [27]:
mars_info['mars_facts_html'] = mars_facts_html

# Mars Hemispheres

Image link for this part of the exercise wasn't working.